# Evaluate a POMDP Policy

In [1]:
rng = MersenneTwister(1);

In [2]:
using AutomotiveDrivingModels
using AutoViz
using GridInterpolations
using POMDPs
using POMDPToolbox
using AutomotivePOMDPs
using QMDP
using SARSOP
using Parameters
using Reel

06-Aug 10:55:41:WARNING:root:replacing docs for 'AutomotivePOMDPs.scene_to_states :: Tuple{Records.Frame{Records.Entity{AutomotiveDrivingModels.VehicleState,AutomotiveDrivingModels.VehicleDef,Int64}},AutomotivePOMDPs.SingleOCPOMDP}' in module 'AutomotivePOMDPs'.
06-Aug 10:55:46:WARNING:root:replacing docs for 'AutomotivePOMDPs.states_to_scene :: Tuple{Dict{Int64,AutomotivePOMDPs.SingleOCState},AutomotivePOMDPs.SingleOCPOMDP}' in module 'AutomotivePOMDPs'.


In [3]:
# include("../src/explicit_pomdps/single_crosswalk/decomposition.jl")

## Define Environment

In [79]:
params = CrosswalkParams(ped_rate=0.05)
env = CrosswalkEnv(params)

06-Aug 11:58:38:WARNING:root:replacing docs for 'AutomotivePOMDPs.CrosswalkEnv :: Union{Tuple{AutomotivePOMDPs.CrosswalkParams}, Tuple{}}' in module 'AutomotivePOMDPs'.


AutomotivePOMDPs.CrosswalkEnv(Roadway, AutomotiveDrivingModels.Lane(LaneTag(2, 1), AutomotiveDrivingModels.CurvePt[CurvePt({25.000, -10.000, 1.571}, 0.000, 0.000, NaN), CurvePt({25.000, 10.000, 1.571}, 20.000, 0.000, NaN)], 6.0, AutomotiveDrivingModels.SpeedLimit(-Inf, Inf), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneBoundary(:unknown, :unknown), AutomotiveDrivingModels.LaneConnection[], AutomotiveDrivingModels.LaneConnection[]), AutomotiveDrivingModels.ConvexPolygon[ConvexPolygon: len 4 (max 4 pts)
	VecE2(15.000, -1.500)
	VecE2(15.000, -4.500)
	VecE2(21.500, -4.500)
	VecE2(21.500, -1.500)
], AutomotivePOMDPs.CrosswalkParams(2, 50.0, 3.0, 20.0, 6.0, 5.0, 37.0, 8.0, 100, 0.05, 2.0, 10.0))

In [50]:
fieldnames(env)

4-element Array{Symbol,1}:
 :roadway  
 :crosswalk
 :obstacles
 :params   

In [24]:
fieldnames(CrosswalkParams)

12-element Array{Symbol,1}:
 :n_lanes         
 :roadway_length  
 :lane_width      
 :crosswalk_length
 :crosswalk_width 
 :start_pos       
 :end_pos         
 :speed_limit     
 :n_max_ped       
 :ped_rate        
 :ped_max_speed   
 :ped_max_y       

## Solve Policy in Discrete, Single Agent Environment

In [95]:
dpomdp = SingleOCPOMDP(env=env, ΔT = 0.5, p_birth = 0.7, pos_res = 1.0, vel_res=1.0)
solver = QMDPSolver(max_iterations=100, tolerance=1e-3, verbose=true)
dpolicy = solve(solver, dpomdp);

[Iteration 1   ] residual:          1 | iteration runtime:   2822.877 ms, (      2.82 s total)
[Iteration 2   ] residual:       0.95 | iteration runtime:   2820.112 ms, (      5.64 s total)
[Iteration 3   ] residual:      0.903 | iteration runtime:   2828.404 ms, (      8.47 s total)
[Iteration 4   ] residual:      0.857 | iteration runtime:   2816.652 ms, (      11.3 s total)
[Iteration 5   ] residual:      0.811 | iteration runtime:   2796.110 ms, (      14.1 s total)
[Iteration 6   ] residual:      0.762 | iteration runtime:   3037.634 ms, (      17.1 s total)
[Iteration 7   ] residual:      0.707 | iteration runtime:   2848.874 ms, (        20 s total)
[Iteration 8   ] residual:      0.649 | iteration runtime:   2807.324 ms, (      22.8 s total)
[Iteration 9   ] residual:      0.553 | iteration runtime:   2893.314 ms, (      25.7 s total)
[Iteration 10  ] residual:      0.506 | iteration runtime:   2927.734 ms, (      28.6 s total)
[Iteration 11  ] residual:      0.457 | iteration 

## Set up High Fidelity Evaluation Environment

In [26]:
include("../src/eval-crosswalk/config.jl")
include("../src/eval-crosswalk/ego_control.jl")
include("../src/eval-crosswalk/sensor.jl")
include("../src/eval-crosswalk/policy.jl")
include("../src/eval-crosswalk/state_estimation.jl")
include("../src/eval-crosswalk/simulation.jl")
include("../src/eval-crosswalk/render_helpers.jl")

In [66]:
config = EvalConfig(time_out = 300, n_episodes = 500)

EvalConfig 
	 Sim step (s) 0.10 
	 N episodes  500 
	 Time out  300 
	 Time out  300 


In [80]:
cam = StaticCamera(VecE2(25, 0.), 20.);

In [81]:
sensor = POMDPSensor(pomdp = dpomdp, sensor = SimpleSensor(0.5, 0.5))

POMDPSensor
  pomdp: AutomotivePOMDPs.SingleOCPOMDP
  sensor: SimpleSensor


In [82]:
fieldnames(SimpleSensor)

2-element Array{Symbol,1}:
 :pos_noise
 :vel_noise

In [83]:
policy = QMDPEval(env, dpomdp, dpolicy);

In [84]:
up = MixedUpdater(dpomdp, config.sim_dt);

In [85]:
a0 = -4.
ego0 = initial_ego(env, config.rng)
b0 = Dict{Int64, SingleOCDistribution}()
b0[-1] = initial_state_distribution(dpomdp, ego0.state)
o0 = Dict{Int64, SingleOCObs}()

Dict{Int64,AutomotivePOMDPs.SingleOCState} with 0 entries

In [86]:
update_freq = 5
ego_model = CrosswalkDriver(update_freq, 0, env, a0, b0, o0, ego0, sensor, policy, up);

In [87]:
reset_model!(ego_model, ego0)

0.0

In [88]:
overlay = QMDPOverlay(ego_model);

In [92]:
env.params.ped_rate = 0.0
ego0 = initial_ego(env, config.rng)
reset_model!(ego_model, ego0)
models = Dict{Int, DriverModel}()
models[1] = ego_model
scene = initial_scene(models, env, config)
push!(scene, ego0)
nticks = 100
rec = SceneRecord(nticks+1, config.sim_dt)

SceneRecord(nscenes=0)

In [93]:
# execute the simulation
@time simulate!(rec, scene, env, models, nticks, config.rng, config.n_ped, config.callbacks)

  1.281409 seconds (14.64 M allocations: 525.537 MiB, 18.91% gc time)


SceneRecord(nscenes=101)

In [94]:
reset_overlay!(overlay, ego0)
duration, fps, render_rec = animate_record(rec, overlay, 24)
speed_factor = 1
film = roll(render_rec, fps = fps*speed_factor, duration = duration/speed_factor)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpuOzfUv", 0x0000000000000065, 10.0, nothing)